In [ ]:
!pip install -q transformers datasets torch sentence_transformers

In [ ]:
!huggingface-cli login --token hf_

In [ ]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification, Trainer
from datasets import load_dataset

# Load the model and tokenizer
model_name = "minoosh/empathy-crossencoder-cross_entropy"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Load the dataset
dataset = load_dataset("minoosh/EPITOME_pairs")

# Tokenize the test split
def preprocess_test_function(examples):
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    return encodings

# Apply tokenization to the test dataset
tokenized_test = dataset['test'].map(preprocess_test_function, batched=True)
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask'])


In [16]:
import torch
from datasets import load_dataset
from transformers import BertForSequenceClassification, AutoTokenizer, AutoConfig, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load dataset
dataset = load_dataset("minoosh/EPITOME_pairs")

# Initialize the tokenizer and model for cross-encoder setup
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    encodings['labels'] = examples['label']  # Add labels
    return encodings

# Apply tokenization
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
tokenized_val = dataset['validation'].map(preprocess_function, batched=True)

# Set format for PyTorch
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define compute_metrics function for classification evaluation
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = predictions.argmax(axis=1)
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    f1 = f1_score(labels, preds, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Custom Cross-Encoder model class for classification
class CrossEncoderModel(torch.nn.Module):
    def __init__(self, model_name, num_classes=4, loss_fn="cross_entropy"):
        super(CrossEncoderModel, self).__init__()
        # Load model config
        self.config = AutoConfig.from_pretrained(model_name, num_labels=num_classes)
        self.model = BertForSequenceClassification.from_pretrained(model_name, config=self.config)
        self.loss_fn = loss_fn

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Output logits for classification

        loss = None
        if labels is not None:
            if self.loss_fn == "cross_entropy":
                loss_fct = torch.nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
                loss = loss_fct(logits, labels)
            elif self.loss_fn == "focal_loss":
                # Focal loss implementation for handling class imbalance
                alpha = 0.25
                gamma = 2.0
                ce_loss = torch.nn.CrossEntropyLoss(reduction="none")(logits, labels)
                pt = torch.exp(-ce_loss)  # Probability of the true class
                loss = (alpha * (1 - pt) ** gamma * ce_loss).mean()
            elif self.loss_fn == "kl_divergence":
                # KL Divergence for soft-label classification
                kl_div = torch.nn.KLDivLoss(reduction="batchmean")
                soft_labels = torch.nn.functional.one_hot(labels, num_classes=self.config.num_labels).float()
                log_probs = torch.nn.functional.log_softmax(logits, dim=-1)
                loss = kl_div(log_probs, soft_labels)
            else:
                raise ValueError(f"Unsupported loss function: {self.loss_fn}")

        return {"loss": loss, "logits": logits}

# Function to initialize and train the cross-encoder model
def test_crossencoder(model_name):
    
    model = CrossEncoderModel(model_name=model_name)

    # Initialize the Trainer
    trainer = Trainer(model=model)
    
    # Make predictions on the test dataset
    predictions = trainer.predict(tokenized_test)
    
    # Get the predicted class indices
    predicted_classes = predictions.predictions.argmax(axis=1)
    
    # If you want to compare with actual labels
    actual_labels = tokenized_test['label']
    
    # Print out predictions and actual labels for verification
    for i in range(len(predicted_classes)):
        print(f"Predicted: {predicted_classes[i]}, Actual: {actual_labels[i]}")

    # Save and push the model to the Hugging Face Hub
    #trainer.save_model(f"./output/empathy-crossencoder-{loss_fn}")
    #trainer.push_to_hub(f"minoosh/empathy-crossencoder-{loss_fn}")
    
    return predicted_classes, actual_labels

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
a,b = test_crossencoder("minoosh/bert_empathy-crossencoder-focal_loss")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at minoosh/bert_empathy-crossencoder-focal_loss and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.

Predicted: 2, Actual: 0
Predicted: 2, Actual: 1
Predicted: 2, Actual: 0
Predicted: 2, Actual: 0
Predicted: 2, Actual: 1
Predicted: 2, Actual: 2
Predicted: 2, Actual: 0
Predicted: 2, Actual: 3
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 0
Predicted: 2, Actual: 0
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 1
Predicted: 2, Actual: 0
Predicted: 2, Actual: 1
Predicted: 2, Actual: 1
Predicted: 2, Actual: 2
Predicted: 2, Actual: 0
Predicted: 2, Actual: 3
Predicted: 2, Actual: 2
Predicted: 2, Actual: 1
Predicted: 2, Actual: 3
Predicted: 2, Actual: 0
Predicted: 2, Actual: 0
Predicted: 2, Actual: 2
Predicted: 2, Actual: 3
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 0
Predicted: 2, Actual: 0
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 2
Predicted: 2, Actual: 3
Predicted: 2, Actual: 2
Predicted: 2, Actual: 3
Predicted: 2, Actual: 1
Predicted: 2, Ac

In [21]:
def compute_metrics(predictions, labels):
    #predictions, labels = eval_pred
    preds = predictions
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    f1 = f1_score(labels, preds, average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

compute_metrics(a,b)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.4155844155844156,
 'precision': 0.17383923266276208,
 'recall': 0.4155844155844156,
 'f1': 0.24513735112813453}

In [14]:
a

{'eval_loss': 1.3570916652679443,
 'eval_runtime': 5.0908,
 'eval_samples_per_second': 60.501,
 'eval_steps_per_second': 3.929}

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
from datasets import load_dataset

# Load the model and tokenizer
#minoosh/bert_empathy-crossencoder-focal_loss
#bert-base-uncased
#minoosh/empathy-crossencoder-cross_entropy
tokenizer = AutoTokenizer.from_pretrained("minoosh/bert_empathy-crossencoder-focal_loss")
model = AutoModelForSequenceClassification.from_pretrained("minoosh/bert_empathy-crossencoder-focal_loss")

# Load the dataset
dataset = load_dataset("minoosh/EPITOME_pairs")

# Tokenize the test split
def preprocess_test_function(examples):
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    return encodings

# Apply tokenization to the test dataset
tokenized_test = dataset['test'].map(preprocess_test_function, batched=True)
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask'])

'''dataset = load_dataset("minoosh/EPITOME_pairs")

# Preprocess data for the cross-encoder model by concatenating text1 and text2 with [SEP]
def preprocess_function(examples):
    # Concatenate both texts with a [SEP] token in between
    encodings = tokenizer(examples['text1'], examples['text2'], truncation=True, padding=True, max_length=512)
    #encodings['labels'] = examples['label']  # Add labels
    return encodings

# Apply tokenization
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)
tokenized_test.set_format(type='torch', columns=['input_ids', 'attention_mask'])
tokenized_val = dataset['validation'].map(preprocess_function, batched=True)'''




# Initialize the Trainer
trainer = Trainer(model=model)

# Make predictions on the test dataset
predictions = trainer.predict(tokenized_test)

# Get the predicted class indices
predicted_classes = predictions.predictions.argmax(axis=1)

# If you want to compare with actual labels
actual_labels = tokenized_test['label']

# Print out predictions and actual labels for verification
for i in range(len(predicted_classes)):
    print(f"Predicted: {predicted_classes[i]}, Actual: {actual_labels[i]}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at minoosh/bert_empathy-crossencoder-focal_loss and are newly initialized: ['classifier.bias', 'classifier.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: minooshayan97 (minoosh). Use `wandb login --relogin` to force relogin


Predicted: 1, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 0
Predicted: 1, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 2
Predicted: 1, Actual: 0
Predicted: 1, Actual: 3
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 0
Predicted: 1, Actual: 0
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 1
Predicted: 1, Actual: 0
Predicted: 1, Actual: 1
Predicted: 1, Actual: 1
Predicted: 1, Actual: 2
Predicted: 1, Actual: 0
Predicted: 1, Actual: 3
Predicted: 1, Actual: 2
Predicted: 1, Actual: 1
Predicted: 1, Actual: 3
Predicted: 1, Actual: 0
Predicted: 1, Actual: 0
Predicted: 1, Actual: 2
Predicted: 1, Actual: 3
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 0
Predicted: 1, Actual: 0
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 2
Predicted: 1, Actual: 3
Predicted: 1, Actual: 2
Predicted: 1, Actual: 3
Predicted: 1, Actual: 1
Predicted: 1, Ac

In [4]:
trainer.evaluate(tokenized_test)

{'eval_runtime': 4.9493,
 'eval_samples_per_second': 62.231,
 'eval_steps_per_second': 4.041}